In [1]:
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import ListedColormap
import seaborn as sns
import numpy as np

In [2]:
main_folder = '/ghds/cv19/analysis'
flowcell_id = '201216_NB552470_0003_AHHJT7BGXG'
path = os.path.join(main_folder, flowcell_id)
df = pd.read_csv(os.path.join(path, 'c19_call.hdr.tsv'), sep='\t')

In [3]:
all_samples = pd.read_csv(os.path.join(path, 'c19_read_counts.hdr.tsv'), sep='\t').dropna(subset=['run_sample_id']).sort_values(by=['pos_tube_rack', 'pos_extraction'])
all_samples['RBP'] = all_samples.pos_tube_rack.str[:13]
all_samples['primer_plate'] = all_samples.pos_extraction.str[:13]
all_samples['row'] = all_samples.pos_extraction.str[14]
all_samples['column'] = all_samples.pos_extraction.str[15:].astype('int')

In [4]:
RBP = ['RBP201112A001', 'RBP201112A228', 'RBP201112A229', 'RBP201112A230']

## 201216_EXP19_SpittoonStudy1
RBP = ['RBP201112A001', 'RBP201112A228', 'RBP201112A229', 'RBP201112A230']

In [ ]:
total_cols = 3
total_rows = 4
fig, axs = plt.subplots(nrows=total_rows, ncols=total_cols, sharex=True, sharey=True, 
                        figsize=(7*total_cols, 5*total_rows), constrained_layout=True, squeeze=False)

# mpl.rcParams['figure.dpi'] = 500

fig.suptitle(f'{flowcell_id}: RNAse Count by RT-PCR Plate, Spittoon Study #1')
fig.set_facecolor('white')

for i,k in zip(RBP, range(4)):
    sub_RBP = all_samples[all_samples['RBP'] == i].sort_values(by='primer_plate')
    
    for l,m in zip(sub_RBP.primer_plate.unique(), range(3)):
        sub_primer = sub_RBP[sub_RBP['primer_plate'] == l]
        sub_primer.rnase_count.astype('int64')
        
        plate_map = sub_primer.iloc[:, [22, 23, 7]].pivot(index='row', columns='column', values='rnase_count').fillna(0)

        for j in range(len(plate_map.columns) + 1, 13):
            plate_map[f'{j}'] = 0
            
        plate_map = plate_map.astype(dtype='int')
        
        row = k % total_rows
        pos = m % total_cols
        
        fig = sns.heatmap(plate_map, annot=True, cbar=False, cmap="Purples", 
                    vmin=500, fmt='d', annot_kws={"fontsize":9}, ax=axs[row][pos])
        
        fig = sns.heatmap(plate_map, mask=plate_map > 50, cmap=ListedColormap(['grey']), annot=False, ax=axs[row][pos], cbar=False)

        axs[row][pos].set_title(f'RBP Plate: {i}, Primer Plate: {l}')
            
# plt.savefig('EXP13_spittoon_aliquot', dpi=300)
